In [ ]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import sys
import math
import glob
import os
import random
import seaborn as sns

# import classifiers
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold

import mne
from mne.preprocessing import EOGRegression
from mne.defaults import HEAD_SIZE_DEFAULT
from mne.channels._standard_montage_utils import _read_theta_phi_in_degrees

# set directories
dir                 = '/home/amber/OneDrive/code/nAdaptation_EEG_git/' # to save figures
dir_data            = 'data/EEG/raw/'
dir_behaviour       = 'data/behaviour/'

# extract eeg files
eeg_files = sorted(glob.glob(dir_data + '*.bdf'))
# print(eeg_files)

# extract behaviour files
behaviour_files = sorted(glob.glob(dir_behaviour + '*.txt'))
# print(behaviour_files)

# determine 
assert len(eeg_files) == len(behaviour_files) # throws error if number of eeg files doesn't match number of behavioural files
n_sub = len(eeg_files)

for i in range(len(eeg_files)):
    assert behaviour_files[i][-9:-4] == eeg_files[i][-9:-4]

# exclude subjects
exclude = []
n_sub = len(eeg_files)

print('\nNumber of subjects: ', n_sub - len(exclude))

In [ ]:
# montage - creates visualization of cap when figures are plotted with mne functions
fname = dir + 'config/chs.tsv'
montage = _read_theta_phi_in_degrees(fname=fname, head_size=HEAD_SIZE_DEFAULT,
                                    fid_names=['Nz', 'LPA', 'RPA'],
                                    add_fiducials=False)

# set reference electrodes
reference       = ['EMGL', 'EMGR']
eog             = ['EOGL', 'EOGR', 'EOGT', 'EOGB']
exc             = ['EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']

In [ ]:
# settings
tmin                    = -0.1
tmax                    = 0.5
sample_rate             = 2048
down_sample_rate        = 256

# band-pass frequencies
f_low       = 0.1           # high-pass filter
f_high      = 30            # low-pass filter

f_notch     = 50            # notch filter

# targets
targets                 = [3, 6, 8, 9]
n_targets               = len(targets)

# set timepoints
n_timepoints            = math.ceil((abs(tmin) + tmax)/(1/down_sample_rate))
t                       = np.arange(n_timepoints)*(1/down_sample_rate)+tmin

# reject criteria to exclude certain epochs based on min/max voltages
# reject_criteria         = dict(eeg=3.6e-4)
# flat_criteria           = dict(eeg=50e-10)
reject_criteria         = None
flat_criteria           = None

# channel info
n_channels      = 64

global channel_names # (e.g. Oz, Fp1) will be added later in the loop over subjects

In [ ]:
# trial types
trial_type_clean                = 'clean'
trial_type_single               = 'single'
trial_type_repeated             = 'repeated'

# number of repetitions
repetitions_clean               = 100 * n_targets
repetitions_single              = 8 * n_targets
repetitions_repeated            = 8 * n_targets

# contrast
contrast_clean                  = ['f_contrast']
contrast_clean_values           = [1]

contrast_single                 = ['l_contrast', 'lm_contrast', 'm_contrast', 'mh_contrast', 'h_contrast']
contrast_single_values          = [0.50, 0.60, 0.70, 0.80, 0.90]

contrast_repeated               = ['l_contrast', 'lm_contrast', 'm_contrast', 'mh_contrast', 'h_contrast']
contrast_repeated_values        = [0.50, 0.60, 0.70, 0.80, 0.90]

# adapter
adapter_clean                   = ['none']
adapter_single                  = ['none']
adapter_repeated                = ['same', 'different']

# image types for which we are going to extract the trigger numbers
img_type_clean                  = ['test']
img_type_single                 = ['test']
img_type_repeated               = ['test']

# concatenate information for the different trial types
trial_types                     = [trial_type_clean, trial_type_single, trial_type_repeated]
repetitions                     = [repetitions_clean, repetitions_single, repetitions_repeated]
contrasts                       = [contrast_clean, contrast_single, contrast_repeated]
contrasts_value                 = [contrast_clean_values, contrast_single_values, contrast_repeated_values]
adapters                        = [adapter_clean, adapter_single, adapter_repeated]
img_types                       = [img_type_clean, img_type_single, img_type_repeated]

In [ ]:
# initiate dataframes
data_clean          = np.zeros((n_sub,      len(contrast_clean),         len(adapter_clean),         repetitions_clean,         n_channels,        n_timepoints))
data_single         = np.zeros((n_sub,      len(contrast_single),        len(adapter_single),        repetitions_single,        n_channels,      n_timepoints))
data_repeated       = np.zeros((n_sub,      len(contrast_repeated),      len(adapter_repeated),      repetitions_repeated,      n_channels,    n_timepoints))

# shape (n_sub x contrast x adapter x channels x timepoints)
print('Shape data struct for clean trials: '.ljust(50), data_clean.shape)
print('Shape data struct for single trials: '.ljust(50), data_single.shape)
print('Shape data struct for repeated trials: '.ljust(50), data_repeated.shape)

data                = [data_clean, data_single, data_repeated]

Below we define two functions that we use when we loop through the subjects

In [ ]:
def triggerNum_extract(file, trial_types, contrasts, adapters):

    # initiate lis to store trigger numbers
    dic_triggerNums = {}

    for iT, trial_type in enumerate(trial_types):

            # initiate nested dictionairy
            dic_trial_type = {}

            for contrast in contrasts[iT]:

                # initiate nested dictionairy
                dic_contrast = {}

                for adapter in adapters[iT]:

                    # select rows
                    idx = file[(file.trial_type == trial_type) & (file.contrast == contrast) & (file.adapter == adapter) & (file.img_type == 'test')].index
                    dic_contrast[adapter] = file.loc[idx, 'trigger_num'].unique()[0]

                # append to dictionairies
                dic_trial_type[contrast] = dic_contrast
            dic_triggerNums[trial_type] = dic_trial_type

    return dic_triggerNums

def create_arrays(sub_index, data_current_subject, events_triggerNums, triggerNums, trial_types, contrasts, adapters):

    for iT, trial_type in enumerate(trial_types):
            for iC, contrast in enumerate(contrasts[iT]):
                for iA, adapter in enumerate(adapters[iT]):

                        # select triggerNum
                        current_triggerNum = triggerNums[trial_type][contrast][adapter]

                        # find indices
                        idx = np.where(np.array(events_triggerNums)==current_triggerNum)[0]

                        print(trial_type, ',', contrast, ',', adapter, ',', len(idx))

                        # add to dataframe
                        data[iT][sub_index, iC, iA, :, :, :] = data_current_subject[idx, :, :]

In [ ]:
# mne.set_log_level('WARNING') # suppresses warnings from MNE (less output)

# iterate over subjects and extract data
count_sub = 0
dropped_epochs = np.zeros(n_sub)
for i in range(n_sub):

    # import behaviour file
    df_behaviour = pd.read_csv(behaviour_files[i])
    triggerNums = triggerNum_extract(df_behaviour, trial_types, contrasts, adapters)

    # import raw data
    raw = mne.io.read_raw_bdf(eeg_files[i], eog=eog, misc=reference, exclude=exc, preload=True)
    channel_names = raw.info['ch_names'][:n_channels]

    # reference signal to mastoids
    raw.set_eeg_reference(reference)

    # apply band-pass and notch filter
    raw.notch_filter(freqs=(f_notch))
    raw.filter(l_freq=f_low, h_freq=f_high)

    # set electrode montage
    raw.set_montage(montage)

    # find events
    events = mne.find_events(raw)
    events = mne.pick_events(events, include=df_behaviour['trigger_num'].unique().tolist().remove(-4))
    events_triggerNums = events[:, 2].tolist() # contains triggernumbers in temporal order)
    
    # create epochs
    if down_sample_rate == None:
        epochs = mne.Epochs(raw, events, baseline=(tmin, 0), picks=['eeg', 'eog'], reject=reject_criteria, flat=flat_criteria, tmin=tmin, tmax=tmax, preload='True')
    else:
        epochs = mne.Epochs(raw, events, baseline=(tmin, 0), picks=['eeg', 'eog'], reject=reject_criteria, flat=flat_criteria, tmin=tmin, tmax=tmax, preload='True').resample(down_sample_rate, npad='auto')

    # EOG regression + baseline correction
    model_plain = EOGRegression(picks='eeg', picks_artifact='eog').fit(epochs)
    epochs_clean_plain = model_plain.apply(epochs)
    epochs_clean_plain.apply_baseline(baseline=(tmin, 0))

    # transform to current source density (CSD)
    epochs_clean_plain_csd = mne.preprocessing.compute_current_source_density(epochs_clean_plain)

    # store data in dataframe
    data_current_subject = epochs_clean_plain_csd.get_data(picks=channel_names)

    # create and save arrays
    create_arrays(i, data_current_subject, events_triggerNums, triggerNums, trial_types, contrasts, adapters)

    # increment count
    count_sub+=1

# save dataframes
np.save(dir + 'data/EEG/data_clean', data[0])
np.save(dir + 'data/EEG/data_single', data[1])
np.save(dir + 'data/EEG/data_repeated', data[2])

In [ ]:
print(channel_names)